To start, call the contents of my package, ignore the warnings or errors, this is because somehow we are redefining something by calling to packages that have the same function

In [1]:
using FractionalKrylovMC

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


We create a random problem

In [2]:
# USAGE: problem = create_random_problem(α::ExponentInTime,γ::ExponentInSpace,k::SizeOfVector)
α = 0.9 # using derivative in time ^C_0 D_t^α
γ = 0.8 # using A^γ as the linear operator
# perhaps lower this!
k = 300 # size(u0) = k; size(A) = (k,k)
problem = create_random_problem(α,γ,k,MittagLefflerRand())

MittagLefflerProblem{Float64}(300, [0.4706190758383446 -0.017590365465258065 … 0.008588930808109579 -0.003751677386833397; -0.01759036546525812 0.4790536367177919 … -0.0037366970144917713 -0.012629021145861605; … ; 0.008588930808109487 -0.003736697014491478 … 0.37308132300291347 -0.016999775281211552; -0.00375167738683336 -0.012629021145861678 … -0.01699977528121083 0.4775340786377503], [0.4706190758383446 -0.017590365465258065 … 0.008588930808109579 -0.003751677386833397; -0.01759036546525812 0.4790536367177919 … -0.0037366970144917713 -0.012629021145861605; … ; 0.008588930808109487 -0.003736697014491478 … 0.37308132300291347 -0.016999775281211552; -0.00375167738683336 -0.012629021145861678 … -0.01699977528121083 0.4775340786377503], [0.8812943823326752, 0.76684033944443, 0.037137237436129955, 0.10049754976607206, 0.08087350542066363, 0.949218904636372, 0.3306678860169868, 0.5299264079076317, 0.4488746965057916, 0.327978316248961  …  0.08697989462937772, 0.8268257906940923, 0.90836444

problem has following fields

In [ ]:
# relevant parameters of the problem
problem.A # matrix
problem.u0 # initial condition
problem.t # time
# we want to compute
# mittleff(α,-A^γ*t^α)*u0

the way I create the problem, I already compute the exact solution

In [ ]:
problem.uT_spectral

we can call the Monte-Carlo solver, which computes the exponential matrix by computing $\exp(-At)$ for all random times $t$ and does not project to any subspace

In [3]:
nsims = Int(1e5)
cutoff = 1e6 # ignoring all times beyond cutoff
MCSol = solve(problem,nsims,cutoff,MCSolver())

FractionalKrylovMC.MittagLefflerMCSolution{Float64}([0.36714449081163797, 0.16054479224034057, 0.7087889003337605, 0.09496610322228217, 0.43542200230018957, 0.21124649674315274, 0.7212987105816101, 0.41959887371719296, 0.49372912924625056, 0.23225349751277044  …  0.3223293753086675, 0.12431296863590438, 0.1019476738761953, 0.8443861179070706, 0.5717494720550758, 0.693510869540203, 0.4737610420673497, 0.11057805280176222, 0.2551225505365903, 0.6557319443299396], [9.018043827341159e-8, 7.624961154888441e-7, 2.642492874926617e-6, 5.366256055021698e-6, 1.5090989019786715e-5, 1.5587282840024563e-5, 1.918102561295016e-5, 2.2303104506077277e-5, 2.3283488152344732e-5, 2.3301475717808258e-5  …  2534.079598013102, 3122.100360232664, 6228.928226374932, 8750.827586839128, 9657.88809282001, 10588.808157937881, 13361.092769772798, 16932.696921599985, 41524.36342069755, 49573.1272526413], 100000, 1.0e6)

Check relative error and random entries to see whether it is correct, checking with confidence ranges will be done in the future

In [4]:
using Random
@show relative_error(problem.uT_spectral,MCSol.uT);
entries_to_show = randperm(k)[1:5]
@show problem.uT_spectral[entries_to_show];
@show MCSol.uT[entries_to_show];

relative_error(problem.uT_spectral, MCSol.uT) = 0.0008520554223556011
problem.uT_spectral[entries_to_show] = [0.12414436853620948, 0.6275463597562488, 0.7879545326946402, 0.8035879390711249, 0.24484002331160093]
MCSol.uT[entries_to_show] = [0.12431296863590438, 0.6281522186445314, 0.788744477496306, 0.8040513603493401, 0.24477503862721262]


can also use Krylov!

In [5]:
KrySubspaceDim = 5
MCKrySol = solve(problem,nsims,cutoff,KrySubspaceDim,MCSolverExpokit())

MittagLefflerMCSolution{Float64}([0.11636792768055922, 0.03590590232145222, 0.09095050889481589, 0.5877801084548403, 0.3312206673144261, 0.5808196195255234, 0.3774191449106153, 0.12705843474498177, 0.2987390918440006, 0.399895148964329  …  0.25722577350422654, 0.07298003390713695, 0.018049700787571336, 0.08802300657120808, 0.7026595843706074, 0.19096146424429045, 0.7175306864704117, 0.22248998464868872, 0.36509778629551143, 0.29578879059167656], [2.2923520499773145e-6, 3.799145958859502e-6, 1.5132093552727634e-5, 1.6060090715816866e-5, 1.7576099950020373e-5, 1.9807230255466673e-5, 2.11699004618462e-5, 2.614284110197647e-5, 2.758549532932784e-5, 2.8661746962706307e-5  …  23523.68249820971, 24975.751542023347, 26856.316512089554, 27225.547935359755, 28895.5865803514, 31755.969661765645, 38945.8723638231, 48549.76383027059, 55477.94722272533, 84481.47328431379], 100000, 1.0e6)

In [6]:
@show relative_error(problem.uT_spectral,MCSol.uT);
entries_to_show = randperm(k)[1:5]
@show problem.uT_spectral[entries_to_show];
@show MCKrySol.uT[entries_to_show];

relative_error(problem.uT_spectral, MCSol.uT) = 0.04405151545783162
problem.uT_spectral[entries_to_show] = [0.32903371631807354, 0.466519351346999, 0.39266344200827225, 0.42292447860218824, 0.3700733660968992]
MCKrySol.uT[entries_to_show] = [0.3435055952725601, 0.48435786832338157, 0.40558008261410644, 0.4253704811065888, 0.3735129089014563]


### Quadrature methods here

In [2]:
α = rand()*0.9 + 0.1; γ = rand()*0.9 + 0.1
k = 30
problem = create_random_problem(α,γ,k,MittagLefflerRand())

MittagLefflerProblem{Float64}(30, [0.5800114343480485 0.08932096781006892 … 0.03492649246162594 0.007448065075231317; 0.08932096781006883 0.4285386652688415 … 0.05861836329718746 0.027656673705123726; … ; 0.03492649246162585 0.05861836329718749 … 0.5059222432690795 -0.0040111218498683305; 0.007448065075231408 0.02765667370512361 … -0.004011121849868439 0.47646191101054375], [0.5800114343480485 0.08932096781006892 … 0.03492649246162594 0.007448065075231317; 0.08932096781006883 0.4285386652688415 … 0.05861836329718746 0.027656673705123726; … ; 0.03492649246162585 0.05861836329718749 … 0.5059222432690795 -0.0040111218498683305; 0.007448065075231408 0.02765667370512361 … -0.004011121849868439 0.47646191101054375], [0.49807051152545545, 0.7817433454280674, 0.9842883341038499, 0.3179394373041289, 0.7235762738271369, 0.7522440756719297, 0.88920507293472, 0.6123300755530039, 0.32128497092156905, 0.5676089701500104  …  0.06342276844988204, 0.5664770255072457, 0.6932771058114134, 0.2192886775631

In [3]:
quad_solution = solve(problem,QuadSolver())

MittagLefflerSolution{Float64}([0.38089957391056034, 0.6045237779390545, 0.7784185015363776, 0.2234210402089807, 0.5540059240054013, 0.552017878452342, 0.6446397385555045, 0.5183877810110208, 0.19658774481444435, 0.38831795052186785  …  0.0676903907396444, 0.4645675858966596, 0.47346607275371577, 0.1705456120735339, 0.2505252885162787, 0.022258870760542896, 0.41382734965955603, 0.043794441789661506, 0.692585570512785, 0.1951400799203437], [0.14624476773029468, 0.22470967973192754, 0.27013101819433744, 0.111281929504097, 0.21731541856179665, 0.24828529779853573, 0.2995505277278444, 0.13414139663971258, 0.14307333305150194, 0.2127712597178583  …  -0.0014039884900382835, 0.1342582281615515, 0.260856235085358, 0.0620905541379366, 0.17022054701330783, 0.03790729968106205, 0.192870302298445, 0.07794981875742044, 0.2666366605843563, 0.11002236352528659], [0.059183105752974606, 0.11578736997119493, 0.20703888752299857, -0.018670405736673296, 0.1103922520156952, 0.04965983227636939, 0.031049566

In [5]:
@show problem.solution
@show quad_solution


problem.solution = MittagLefflerSolution{Float64}([0.38037148476666305, 0.6038091735206103, 0.7776953725010782, 0.2228522706920777, 0.5531318753512809, 0.5507877928268501, 0.6431290933566405, 0.5184321751145082, 0.19555944402027078, 0.387057311039081, -0.012355942356364107, 0.38020399961713436, 0.3827062595803604, 0.6514500924542376, 0.44310481408240604, 0.05510834831568359, 0.5125703123481657, 0.6762019570637867, 0.6198143634568503, 0.6816566517210131, 0.06799015501859335, 0.46449743907243635, 0.4718744431594062, 0.1703139409784299, 0.24930593216889568, 0.021993165962931924, 0.4128477724620958, 0.04301727808736097, 0.6915974278429374, 0.1944175124312453], [0.14621983559745655, 0.2247150631579269, 0.27016429089271005, 0.11126499632630518, 0.21733974205268072, 0.24829239594042912, 0.2995656738569567, 0.13415354652663297, 0.14308123135195944, 0.21278621694424618, 0.04006676778635864, 0.20972605976133238, 0.1652132689029001, 0.2091435246980647, 0.1830622505447734, 0.09467383369493204, 0.1

MittagLefflerSolution{Float64}([0.38089957391056034, 0.6045237779390545, 0.7784185015363776, 0.2234210402089807, 0.5540059240054013, 0.552017878452342, 0.6446397385555045, 0.5183877810110208, 0.19658774481444435, 0.38831795052186785  …  0.0676903907396444, 0.4645675858966596, 0.47346607275371577, 0.1705456120735339, 0.2505252885162787, 0.022258870760542896, 0.41382734965955603, 0.043794441789661506, 0.692585570512785, 0.1951400799203437], [0.14624476773029468, 0.22470967973192754, 0.27013101819433744, 0.111281929504097, 0.21731541856179665, 0.24828529779853573, 0.2995505277278444, 0.13414139663971258, 0.14307333305150194, 0.2127712597178583  …  -0.0014039884900382835, 0.1342582281615515, 0.260856235085358, 0.0620905541379366, 0.17022054701330783, 0.03790729968106205, 0.192870302298445, 0.07794981875742044, 0.2666366605843563, 0.11002236352528659], [0.059183105752974606, 0.11578736997119493, 0.20703888752299857, -0.018670405736673296, 0.1103922520156952, 0.04965983227636939, 0.031049566

In [9]:
@show problem.solution.uT
@show quad_solution.uT

problem.solution.uT = [0.38037148476666305, 0.6038091735206103, 0.7776953725010782, 0.2228522706920777, 0.5531318753512809, 0.5507877928268501, 0.6431290933566405, 0.5184321751145082, 0.19555944402027078, 0.387057311039081, -0.012355942356364107, 0.38020399961713436, 0.3827062595803604, 0.6514500924542376, 0.44310481408240604, 0.05510834831568359, 0.5125703123481657, 0.6762019570637867, 0.6198143634568503, 0.6816566517210131, 0.06799015501859335, 0.46449743907243635, 0.4718744431594062, 0.1703139409784299, 0.24930593216889568, 0.021993165962931924, 0.4128477724620958, 0.04301727808736097, 0.6915974278429374, 0.1944175124312453]
quad_solution.uT = [0.38089957391056034, 0.6045237779390545, 0.7784185015363776, 0.2234210402089807, 0.5540059240054013, 0.552017878452342, 0.6446397385555045, 0.5183877810110208, 0.19658774481444435, 0.38831795052186785, -0.011817633616871584, 0.38144363193844877, 0.3834400419379736, 0.6518261344503671, 0.44384501084184963, 0.056028869534991387, 0.5128051703443

30-element Vector{Float64}:
 0.38089957391056034
 0.6045237779390545
 0.7784185015363776
 0.2234210402089807
 0.5540059240054013
 0.552017878452342
 0.6446397385555045
 0.5183877810110208
 0.19658774481444435
 0.38831795052186785
 ⋮
 0.4645675858966596
 0.47346607275371577
 0.1705456120735339
 0.2505252885162787
 0.022258870760542896
 0.41382734965955603
 0.043794441789661506
 0.692585570512785
 0.1951400799203437